In [1]:
import pdb # needs conda install
import os # base python
import pickle # base python
import json # base python
import pandas as pd # needs conda install
import numpy as np # needs conda install
from scipy.stats import ttest_rel,ttest_ind, ttest_1samp # needs conda install
from matplotlib import pyplot as plt # needs conda install
import seaborn as sns # needs conda install
import math # base python
from sklearn.metrics import adjusted_rand_score # needs conda install

sns.set_context('talk')
%matplotlib inline
plt.rcParams['font.size'] = 22


from humanUtils import * # no conda needed
from cswHumanDatabase import load_final_df, load_dfs # no conda needed
from analysis import * # no conda needed
from utils import * # no conda needed

condL = ['blocked','interleaved','early','middle','late']
%load_ext autoreload
%autoreload 2

# model dfs

## read gridsearch results, rerun and save top model experiment
* run experiments with best params of each gridsearch
* saveout acc, exp and zt of each

## sims info
* version 5: skipt1 = false; no variability across seeds 
* version 1: skipt1 = true; no variability across seeds
* version 2: skipt1 = true; variable concentration across seeds 
* version 9: skipt1 = true starting in 2nd block for blocked, always false for interleaved; variable concentration across seeds

In [2]:
def get_gs_info(gsname):
  # concentration variance
  if (gsname == V9) or (gsname == V2):
    concentration_sd = 0.3
    concentration_lb = 0
    concentration_ub = np.inf
    concentration_mean = args["sch"]["concentration"]
    concentration_info = {
      'concentration_mean': concentration_mean,
      'concentration_sd': concentration_sd,
      'concentration_lb': concentration_lb,
      'concentration_ub': concentration_ub
    }
  else:
    concentration_info = None
  # run fun  
  if gsname == V9:
    run_fn = run_batch_exp_curr_v8and9
  else:
    run_fn = run_batch_exp_curr
  return {"concentration_info":concentration_info,'run_fn':run_fn}
  

In [3]:
V1 = 'gs0317'
V2 = 'gs0317_concentrationAcross'
V9 = 'gs0317_version9_concentrationAcross_skipt1False_except_blocked'
V5 = 'gs0317_version5_skipt1False'
gsnameL = [V5,V1,V2,V9]
gsnameL = [V9]

In [5]:
recompute = True
ns=1

for gsname in gsnameL:
  # read gridsearch dataframe
  gsdf = pd.read_csv(
    '~/wd/csw/interleaved_blocked_modeling/data/all_csv/%s-summdf.csv'%gsname
  ).astype(float)
  
  # find param of best model
  TOP_K = 1
  gsdf.loc[:,"mse-bi"] = gsdf.b_mse1+gsdf.b_mse2+gsdf.i_mse1+gsdf.i_mse2
  topKgsdf = gsdf.sort_values('mse-bi').iloc[:TOP_K]
  args,paramstr = get_argsD(topKgsdf.iloc[0])
  print(gsname)
  print()
  print(paramstr)
  print()
  print(topKgsdf.loc[:,("b_mse1","b_mse2","i_mse1","i_mse2","mse-bi")].round(4).to_dict())
  
  
  
  # run experiment
  if recompute:
    print(gsname)
    # gridsearch params: alpha variance, run fun
    gs_params = get_gs_info(gsname)
    concentration_info = gs_params['concentration_info']
    run_fn = gs_params['run_fn']
    # run
    (exp_batch_data, alpha_per_seed,_,_) = run_fn(ns,args,condL,concentration_info) # [curr],[seeds],{data}
    # unpack and save
    exp = np.stack([exp_batch_data[c][s]['exp'] for c in range(5) for s in range(ns)]).reshape(5,ns,200,5)
    zt = np.stack([exp_batch_data[c][s]['zt'] for c in range(5) for s in range(ns)]).reshape(5,ns,200,5)
    xth = np.stack([exp_batch_data[c][s]['xth'].argmax(-1) for c in range(5) for s in range(ns)]).reshape(5,ns,200,5)
    acc = unpack_acc(exp_batch_data,mean_over_tsteps=True)
#     np.save(f'data_csv/model/{gsname}-acc',acc)
#     np.save(f'data_csv/model/{gsname}-exp',exp)
#     np.save(f'data_csv/model/{gsname}-zt',zt)
#     np.save(f'data_csv/model/{gsname}-xth',xth)
#     np.save(f'data_csv/model/{gsname}-alpha',np.array(alpha_per_seed))
    

gs0317_version9_concentrationAcross_skipt1False_except_blocked

concentration_4.055-stickiness_wi_5.715-stickiness_bt_5.715-sparsity_0.343-pvar_0.000-lrate_1.000-lratep_1.000-decay_rate_1.000-beta2_0.000-skipt1_0.000-ppd_allsch_0.000

{'b_mse1': {6668: 0.0104}, 'b_mse2': {6668: 0.0057}, 'i_mse1': {6668: 0.0106}, 'i_mse2': {6668: 0.009}, 'mse-bi': {6668: 0.0358}}
gs0317_version9_concentrationAcross_skipt1False_except_blocked


In [17]:
exp_batch_data[-1][0]['prior'][:,:,0]

array([[  3.95240567,   6.71467038,   7.71467038,   8.71467038,
         -1.        ],
       [  9.71467038,   4.        ,   4.        ,   4.        ,
         -1.        ],
       [  4.        ,  10.71467038,  11.71467038,  12.71467038,
         -1.        ],
       [ 13.71467038,   8.        ,  14.71467038,  15.71467038,
         -1.        ],
       [ 16.71467038,  17.71467038,  18.71467038,  19.71467038,
         -1.        ],
       [ 20.71467038,  15.        ,  21.71467038,  22.71467038,
         -1.        ],
       [ 23.71467038,  24.71467038,  25.71467038,  26.71467038,
         -1.        ],
       [ 27.71467038,  22.        ,  28.71467038,  29.71467038,
         -1.        ],
       [ 30.71467038,  31.71467038,  32.71467038,  33.71467038,
         -1.        ],
       [ 34.71467038,  29.        ,  35.71467038,  36.71467038,
         -1.        ],
       [ 37.71467038,  38.71467038,  39.71467038,  40.71467038,
         -1.        ],
       [ 41.71467038,  36.        ,  36.   

In [8]:
zt[-1,0]

array([[ 0.,  0.,  0.,  0., -1.],
       [ 1.,  1.,  1.,  1., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  1.,  1.,  1., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  1.,  1.,  1., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  1.,  1.,  1., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  1.,  1.,  1., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 1.,  

## make accuracy dataframe
* load saved np files from all gridsearches
* extract accuracy and format dataframe

In [5]:
L = []
for gsname in [V5,V1,V2,V9]:
  acc = np.load(f"data_csv/model/{gsname}-acc.npy")
  alpha = np.load(f"data_csv/model/{gsname}-alpha.npy")
  for cix in range(5):
    for six in range(100):
      for tix in range(200):
        if len(alpha[0]):
          a = alpha[cix,six]  
        else:
          a = []
        L.append({
          'gs':gsname,
          'alpha':a,
          'cond':cix,
          'seed':six,
          'trial':tix,
          'acc':acc[cix,six,tix]
        })
model_acc_df = pd.DataFrame(L)
model_acc_df.loc[:,'test'] = model_acc_df.trial >= 160

In [6]:
# model_acc_df.to_csv(f"data_csv/model/acc_df.csv")

## make states dataframe
* as above

In [7]:
L = []
for gsname in [V5,V1,V2,V9]:
  exp = np.load(f"data_csv/model/{gsname}-exp.npy")
  zt = np.load(f"data_csv/model/{gsname}-zt.npy")
  xth = np.load(f"data_csv/model/{gsname}-xth.npy")
  alpha = np.load(f"data_csv/model/{gsname}-alpha.npy")
  for cix in range(5):
    for six in range(100):
      for trix in range(200):
        for tstep in range(5):
          if len(alpha[0]):
            a = alpha[cix,six]  
          else:
            a = []
          L.append({
            'gs':gsname,
            'alpha':a,
            'cond':cix,
            'seed':six,
            'trial':trix,
            'tstep':tstep,
            'exp':exp[cix,six,trix,tstep],
            'zt':zt[cix,six,trix,tstep],
            'xth':xth[cix,six,trix,tstep]
          })

model_states_df = pd.DataFrame(L)


In [8]:
# model_states_df.to_csv(f"data_csv/model/states_df.csv")